In [7]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from pathlib import Path
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
pd.set_option('display.float_format', '{:.6f}'.format)
start_time = time.time()

# Import Model

In [11]:
jupyter_dir = os.path.abspath(str(Path()))

In [12]:
models_path = os.path.join(jupyter_dir,'trained_models','trained_models')
models = os.listdir(models_path)
for i, path in enumerate(models):
    print(f"{i}   {path}")

0   2024_10_09 14-28-45 Random Forest OnlyOuts
1   2024_10_09 14-58-00 Random Forest OnlyOuts
2   2024_10_09 16-32-20 Deep Neural Network OnlyOuts
3   trained_models_old


In [13]:
model = models[2]
model

'2024_10_09 16-32-20 Deep Neural Network OnlyOuts'

In [14]:
model_path = os.path.join(models_path,model)

In [15]:
model_files = os.listdir(model_path)
model_files

['2024_10_09 16-32-20 Deep Neural Network OnlyOuts insample.csv',
 '2024_10_09 16-32-20 Deep Neural Network OnlyOuts outsample.csv',
 '2024_10_09 16-32-20 Deep Neural Network OnlyOuts.pkl',
 '2024_10_09 16-32-20 Deep Neural Network OnlyOuts.txt']

In [16]:
spec_path = os.path.join(model_path,[file for file in model_files if file.endswith('.txt')][0])

In [17]:
with open(spec_path, "r") as specs:
    for line in specs:
        print(line.strip())

Wed Oct  9 16:30:03 2024
spot(s):
[1214.91 1228.34 1235.03 1240.01 1244.81 1245.64 1248.97 1251.98 1253.33
1253.75 1257.85 1259.82 1260.39 1260.44 1260.47 1261.75 1263.07 1263.23
1266.66 1267.3  1273.68 1274.24 1276.78 1276.84 1279.06 1279.51 1281.9
1283.57 1284.14 1284.21 1285.13 1288.77 1289.26 1289.99 1293.41 1295.65
1298.46 1304.46 1305.09 1308.8  1311.13 1314.51 1314.81 1315.19 1317.94
1318.45 1320.45 1322.21 1322.24 1325.42 1325.9  1326.38 1326.79 1328.62
1329.65 1329.75 1330.53 1330.91 1331.14 1331.49 1333.24 1333.44 1333.8
1334.79 1335.63 1336.97 1337.24 1338.18 1338.73 1339.01 1340.15 1340.92
1342.48 1342.83 1348.8  1348.82 1348.83 1349.69 1349.91 1350.78 1351.52
1352.69 1353.69 1353.73 1354.63 1354.95 1358.24 1359.88 1360.   1360.03
1360.47 1360.48 1362.14 1363.84 1365.56 1365.65 1366.54 1367.41 1367.69
1369.22 1369.79 1370.68 1371.44 1372.41 1372.91 1374.2  1376.17 1376.18
1376.5  1377.07 1377.59 1377.99 1379.83 1380.11 1380.72 1380.79 1381.38
1385.23 1385.42 1385.74 1386.77

In [18]:
model_path

'E:\\git\\machine-learning-option-pricing\\trained_models\\trained_models\\2024_10_09 16-32-20 Deep Neural Network OnlyOuts'

In [19]:
model_files

['2024_10_09 16-32-20 Deep Neural Network OnlyOuts insample.csv',
 '2024_10_09 16-32-20 Deep Neural Network OnlyOuts outsample.csv',
 '2024_10_09 16-32-20 Deep Neural Network OnlyOuts.pkl',
 '2024_10_09 16-32-20 Deep Neural Network OnlyOuts.txt']

In [ ]:
outsample_path = os.path.join(model_path,[file for file in model_files if file.find('outsample') != -1][0])
outsample = pd.read_csv(outsample_path).iloc[:,1:]
outsample

# Test Model

In [ ]:
maturities = outsample['days_to_maturity'].unique().tolist()
barrier_type_names = outsample['barrier_type_name'].unique().tolist()
ws = outsample['w'].unique().tolist()
print(f"\nmaturities:\n     maturities = {maturities}")
print(f"\ntypes:\n     barrier_type_names = {barrier_type_names}\n     ws = {ws}\n")

In [ ]:
by_wbt = outsample.copy().set_index(['w','barrier_type_name','days_to_maturity']).sort_index()

In [ ]:
RMSE = pd.DataFrame(
    np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
    columns = barrier_type_names,
    index = maturities
)

for t in maturities:
    for b in barrier_type_names:
        RMSE.loc[t,b] = np.sqrt(np.average((by_wbt.loc[(slice(None), b, t),'outofsample_error'])**2))
print(f"\nRMSE:\n{RMSE}")

In [ ]:
MAE = pd.DataFrame(
    np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
    index = maturities,
    columns = barrier_type_names
)
for t in maturities:
    for b in barrier_type_names:
        MAE.loc[t,b] = np.average(np.abs(by_wbt.loc[(slice(None), b, t),'outofsample_error']))
print(f"\nMAE:\n{MAE}")

In [ ]:
plt.figure()
for b in barrier_type_names:
    plt.plot(MAE.loc[:, b], label=b)
plt.xlabel('maturities in days')
plt.ylabel('RMSE')
plt.legend()
plt.show()
plt.close()

In [ ]:
errors_bypayoff = {}
for w in ws:
    errors = {}
    print(
        f"\n         {'#'*(len(w)+5)}"
        f"\n         # {w}s #"
        f"\n         {'#'*(len(w)+5)}"
    )
    RMSE = pd.DataFrame(
        np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
        columns = barrier_type_names
        )
    RMSE[w] = maturities
    RMSE = RMSE.set_index(w,drop=True)
    for t in maturities:
        for b in barrier_type_names:
            RMSE.loc[t,b] = np.sqrt(np.average((by_wbt.loc[(w, b, t),'outofsample_error'])**2))
    print(f"RMSE:\n{RMSE}")
    errors['RMSE'] = RMSE
    


    MAE = pd.DataFrame(
        np.zeros((len(maturities),len(barrier_type_names)),dtype=float),
        columns = barrier_type_names
    )
    MAE[w] = maturities
    MAE = MAE.set_index(w,drop=True)
    for t in maturities:
        for b in barrier_type_names:
            MAE.loc[t,b] = np.average(np.abs(by_wbt.loc[(w, b, t),'outofsample_error']))
    print(f"MAE:\n{MAE}")
    errors['MAE'] = MAE

    errors_bypayoff[w] = errors
    print("\n\n")

In [ ]:
barrier_type_names

In [ ]:
for w, errors in errors_bypayoff.items():
    for b in barrier_type_names: 
        print(errors['MAE'])

In [ ]:
plt.figure()
for w, errors in errors_bypayoff.items():
    for b in barrier_type_names: 
        MAE = errors['MAE'][b]
        plt.plot(
            MAE.sort_values(), label=f"{b} {w}s ",
        )
plt.legend(loc='best')
plt.show()
plt.close()

In [ ]:
end_time = time.time()
runtime = end_time - start_time
print(f"cpu: {runtime}")